# BERT for Sequence Classification

The previous blog, *Finetuning BERT: a Simple Introduction* explained what finetuning a BERT model actually means and how it can be used to perform well on our own datasets and NLP tasks. Downstream NLP tasks are specific problems that follow a certain structure/format, such as question and answering, named entity recognition, text summarization and article classification. 

My next few blogs will provide specific examples of these NLP tasks and datasets and detail the architectures used to solve these problems. This blog, *BERT for Sequence Classification* will examine how to finetune BERT on a sequence classification task using a news article classification dataset. The ideas from this blog can be extended to any business problem which involves classifying a passage of text. 

In [1]:
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForQuestionAnswering, AutoTokenizer, \
AutoModelForQuestionAnswering, AutoConfig, AutoModel, AutoModelForSequenceClassification, \
AutoModelForTokenClassification
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import pipeline
from transformers.pipelines import question_answering
import tensorflow as tf
import numpy as np
import pandas as pd
import torch
from torchinfo import summary
import resource

## Sequence Classification

Sequence classification is the task of predicting the class or category of a passage of text. Examples of this include predicting the sentiment of a restaurant review or the business type of a document (e.g. HR document, tax document). The input is a text, the output is a probability vector where each entry indicates the likelihood of the respective class. 

GLUE, the General Language Understanding Evaluation benchmark (https://gluebenchmark.com/) is a collection of resources for training, evaluating, and analyzing natural language understanding systems. It is the dataset that highlighted the exceptional performance of BERT. Among the many datasets were sequence classification tasks: 
- glue/sst2" The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.
- glue/mrpc: The Microsoft Research Paraphrase Corpus (Dolan & Brockett, 2005) is a corpus of sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

In business, there are also countless applications. For example, predicting the sentiment of company announcements might give an indication to how share price will react. Or predicting sentiment of restaurant reviews might help consumers identify/pinpoint what exactly is good or bad about the restaurant. 

The dataset we will be using to explore sequence classification is the AG news dataset. AG is a collection of more than 1 million news articles. These articles have been gathered from more than 2000 news sources by ComeToMyHeadm ab academic news search engine which has been running since July, 2004. The task is to classify each of these news articles into 4 topics: 
<br>
{0: world, 1: sports, 2: business, 3: sci/tech}

Below are some example news articles from AG news. 

In [3]:
ds = tfds.load('ag_news_subset', data_dir= 'data/')
ex_input = list(ds['train'].take(25))[-5:]

for d in ex_input:
    print(d['description'].numpy().decode('utf-8'))
    print()

INFO:absl:Load dataset info from data/ag_news_subset/1.0.0
INFO:absl:Reusing dataset ag_news_subset (data/ag_news_subset/1.0.0)
INFO:absl:Constructing tf.data.Dataset ag_news_subset for split None, from data/ag_news_subset/1.0.0


The Fighting Irish opened the season Saturday night with a 20-17 loss at Brigham Young, a performance that doesn #39;t give much hope for a turnaround season.

By Hugh Williamson in Berlin and Richard Milne in Rsselsheim. The mood at Opel #39;s Rssel-heim factory near Frankfurt was as chilly yesterday as the weather.

NEW YORKIBM said yesterday it is in talks to settle a mammoth lawsuit alleging a pension plan adopted by the firm in the 1990s discriminated against 140,000 older workers.

By wire services. Now it #39;s Roy Oswalt  #39;s turn to try to pitch the Astros to their first playoff series victory. He can only hope for more help from his bullpen than Roger Clemens got Sunday.

Sony BMG - aka  #39;Bony #39; - the merged music label is in talks with Grokster, the P2P software company has confirmed. Negotiations are believed to be focused on the development of a new, label-friendly P2P network.



Let's load the BERT-BASE-UNCASED model from Huggingface which has been finetuned on AG NEWS. As the BERT model (loaded from *nateraw/bert-base-uncased-ag-news*) has already been finetuned on our specific dataset, it already performs the task well. Finetuning means that the model has been trained on via gradient descent using labelled samples from AG news. For more details of this, check out the first blog. The BERT sequence classification model is made up of two components:

- `BertModel`: This is the BERT encoder model. It produces contextualized word embeddings for all the tokens in the input sentence and then averages them together to produce a sentence embedding of dimension 768. Each dimension of this embedding represents some abstract features of the sentence, such as the number of adjectives, whether it’s in present tense or it’s sentiment.
- `Linear`: The linear classification layer of dimension 4 that is connected to the pooled outputs. The output of this layer is a probability vector over the 4 news categories (world, sport, business, sci/tech). Note this layer is specific to our downstream NLP task of news classification.

In [4]:
config = AutoConfig.from_pretrained('nateraw/bert-base-uncased-ag-news')
tokenizer = AutoTokenizer.from_pretrained('nateraw/bert-base-uncased-ag-news')
model = AutoModelForSequenceClassification.from_pretrained('nateraw/bert-base-uncased-ag-news', config = config)

Below are the results for the other articles. As the model has already been finetuned on AG news, the results are all correct. 

In [7]:
for d in ex_input:
    art = d['description'].numpy().decode('utf-8')
    labs = list(config.id2label.values())
    actual_class = labs[d['label'].numpy()]
    inp = tokenizer(art, return_tensors='pt')
    with torch.no_grad():
        out = torch.softmax(model(**inp).logits, dim = -1)
    class_ind = np.argmax(out[0])
    predicted_class = labs[class_ind]
    output_dict = {k:v for k, v in zip(config.id2label.values(), out[0])}
    print('Article: {}\nActual class: {}\nPredictions: {}\n'.format(art, actual_class, output_dict))

Article: The Fighting Irish opened the season Saturday night with a 20-17 loss at Brigham Young, a performance that doesn #39;t give much hope for a turnaround season.
Actual class: Sports
Predictions: {'World': tensor(0.0006), 'Sports': tensor(0.9977), 'Business': tensor(0.0010), 'Sci/Tech': tensor(0.0008)}

Article: By Hugh Williamson in Berlin and Richard Milne in Rsselsheim. The mood at Opel #39;s Rssel-heim factory near Frankfurt was as chilly yesterday as the weather.
Actual class: Business
Predictions: {'World': tensor(0.3490), 'Sports': tensor(0.0344), 'Business': tensor(0.5838), 'Sci/Tech': tensor(0.0329)}

Article: NEW YORKIBM said yesterday it is in talks to settle a mammoth lawsuit alleging a pension plan adopted by the firm in the 1990s discriminated against 140,000 older workers.
Actual class: Business
Predictions: {'World': tensor(0.0014), 'Sports': tensor(0.0004), 'Business': tensor(0.9966), 'Sci/Tech': tensor(0.0017)}

Article: By wire services. Now it #39;s Roy Oswalt